In [1]:
import pandas as pd
# package contenant la classe Syllogisme définie dans le cadre du projet 
# qui permet de manipuler les syllogismes et les formats des bdd.
import syllogism as sy 

from utils import table_de_verite, choice_form, implicatur

In [2]:
###----------- Import des bases

df1 = pd.read_csv("../data/ressources/Ragni2016.csv")
df2 = pd.read_csv("../data/ressources/Veser2018.csv")

In [3]:
###----------- Preparation des données en vue de l'analyse

#########----- Création de l'id unique regroupant id et sequence
df1 = df1.assign(id_seq=lambda df: df1.id.apply(str).str.cat(df1.sequence.apply(str),sep="_") + "_R")
df2 = df2.assign(id_seq=lambda df: df2.id.apply(str).str.cat(df2.sequence.apply(str),sep="_") + "_V")


#########----- Concaténation des deux tables

df_concat = pd.concat([df1,df2])
df_concat = df_concat.drop(['id', 'sequence'], axis=1)

#########----- Pour ne pas entrainer deux fois un même syllogisme, on supprimer les doublons
df_drop = df_concat.drop_duplicates(subset=['task','choices'])


#########----- Transformation des données en phrases valides:
df_drop['sentenced'] = df_drop.task.apply(lambda x : sy.Syllogism(x).sentenced)
df_drop['choice_str'] = df_drop[["task","choices"]].apply(lambda x: sy.Syllogism(x[0]).choice_to_str(x[1]), axis=1)

#########----- Selection des colonnes
df_to_train = df_drop[["id_seq","sentenced","choice_str"]]

print(df_to_train.shape)
#########----- enregristement de la base préparée
#df_to_train.to_csv("./data/intermediate/df_to_train.csv" ,index=False)

/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_96995/1922962863.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['sentenced'] = df_drop.task.apply(lambda x : sy.Syllogism(x).sentenced)


(6506, 3)


/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_96995/1922962863.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['choice_str'] = df_drop[["task","choices"]].apply(lambda x: sy.Syllogism(x[0]).choice_to_str(x[1]), axis=1)


In [4]:
# ###----------- Création de la base de données pour l'analyse des résultats
df_drop['task_form'] = df_drop.task.apply(lambda x : sy.Syllogism(x).full_form)
df_drop['human_response'] = df_drop[["task","response"]].apply(lambda x: sy.Syllogism(x[0]).evaluate_conclusion(x[1]), axis=1)
df_drop['has_conclusion'] = df_drop.task.apply(lambda x : sy.Syllogism(x).has_conclusion)

/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_96995/3989021166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['task_form'] = df_drop.task.apply(lambda x : sy.Syllogism(x).full_form)
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_96995/3989021166.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['human_response'] = df_drop[["task","response"]].apply(lambda x: sy.Syllogism(x[0]).evaluate_conclusion(x[1]), axis=1)
/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/

In [5]:
def index_conclusion(task_form):
    
    index_ccl_list=[]
    # on parcourt les réponses possibles
    for i in choice_form:
        # on parcourt les bonnes réponses
        for ccl in table_de_verite[task_form]:
            if ccl == "NVC":
                return [8]
            elif i == ccl:
                index_ccl_list.append(choice_form.index(i))
    
    return index_ccl_list

df_drop['index_conclusion'] = df_drop.task_form.apply(lambda x : index_conclusion(x))


/var/folders/py/4rt338cj1ks4z3332m7l3f880000gp/T/ipykernel_96995/1662319904.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['index_conclusion'] = df_drop.task_form.apply(lambda x : index_conclusion(x))


In [6]:
df_drop = df_drop[["id_seq", "task_form","human_response","has_conclusion","index_conclusion","choice_str"]]
df_drop.head()

,id_seq,task_form,human_response,has_conclusion,index_conclusion,choice_str
0,1_0_R,IA4,"(Iac, True)",True,"[2, 5]","[All managers are clerks, All clerks are manag..."
1,1_1_R,EA3,"(Eac, True)",True,"[4, 5, 6, 7]","[All divers are linguists, All linguists are d..."
2,1_2_R,AI2,"(Ica, True)",True,"[2, 3]","[All climbers are skaters, All skaters are cli..."
3,1_3_R,AA1,"(Aac, True)",True,"[0, 2, 3]","[All bankers are teachers, All teachers are ba..."
4,1_4_R,OA4,"(Oca, True)",True,[5],"[All opticians are actuaries, All actuaries ar..."


In [8]:
df_drop['implicatur'] = df_drop.task_form.apply(lambda x : True if x in implicatur else False)

In [13]:
df_drop.to_csv("../data/intermediate/df_conclusion.csv" ,index=False)